In [ ]:
# %% [markdown]
# # 🌿 Sustainable Smart City Assistant
# *Enhanced UI Version with Beautiful Input/Output Boxes*

# %%
# Install required libraries
!pip install gradio transformers pandas openpyxl
!pip install pyngrok


In [ ]:
# %%
import gradio as gr
import pandas as pd
import random
import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from pyngrok import ngrok

In [ ]:
# Enhanced CSS for beautiful input/output boxes
css = """
.gradio-container {
    font-family: 'Poppins', sans-serif !important;
    background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%) !important;
}
.header {
    background: linear-gradient(135deg, #4CAF50, #2196F3);
    color: white;
    padding: 20px;
    border-radius: 15px;
    text-align: center;
    margin-bottom: 20px;
    box-shadow: 0 8px 16px rgba(0,0,0,0.1);
}
.tab {
    background: rgba(255,255,255,0.9) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    box-shadow: 0 4px 12px rgba(0,0,0,0.05) !important;
    border: 1px solid rgba(0,0,0,0.05) !important;
}

/* Input boxes */
input[type="text"], textarea, select {
    background: rgba(255, 255, 255, 0.95) !important;
    border-radius: 12px !important;
    padding: 15px !important;
    border: 2px solid #4CAF50 !important;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1) !important;
    transition: all 0.3s ease !important;
    font-size: 16px !important;
    color: #333 !important;
}
input[type="text"]:hover, textarea:hover, select:hover {
    border-color: #2196F3 !important;
    box-shadow: 0 0 5px rgba(33, 150, 243, 0.5) !important;
}
input[type="text"]:focus, textarea:focus, select:focus {
    border-color: #FF9800 !important;
    box-shadow: 0 0 5px rgba(255, 152, 0, 0.5) !important;
    outline: none !important;
}

/* Output boxes */
.output-text {
    background: rgba(255, 255, 255, 0.95) !important;
    border-radius: 12px !important;
    padding: 18px !important;
    border: 2px solid #4CAF50 !important;
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05) !important;
    transition: all 0.3s ease !important;
    font-size: 16px !important;
    color: #333 !important;
    font-weight: 400 !important;
}
.output-text:hover {
    border-color: #2196F3 !important;
    box-shadow: 0 4px 12px rgba(33, 150, 243, 0.5) !important;
}

/* Buttons */
button {
    background: linear-gradient(135deg, #4CAF50, #2196F3) !important;
    color: white !important;
    border: none !important;
    border-radius: 25px !important;
    padding: 12px 24px !important;
    font-size: 16px !important;
    font-weight: 500 !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
    margin-top: 10px !important;
}
button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15) !important;
}
button:active {
    transform: translateY(1px) !important;
}

/* File inputs */
.upload-box {
    border: 2px dashed #4CAF50 !important;
    border-radius: 15px !important;
    background: rgba(255, 255, 255, 0.8) !important;
    transition: all 0.3s !important;
}
.upload-box:hover {
    border-color: #2196F3 !important;
    background: rgba(255, 255, 255, 0.9) !important;
}

/* Tab headers */
.tab-nav {
    border-radius: 12px !important;
    margin-bottom: 15px !important;
}
.tab-nav button {
    transition: all 0.3s !important;
}
"""



In [ ]:
# Initialize feedback storage
feedback_data = []



In [ ]:
# Initialize the model (using GPT-2 for Colab compatibility)
model_name = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)



In [ ]:
# Function to generate responses using the model
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to provide waste management instructions
def waste_management(item):
    prompt = f"Provide detailed disposal instructions for {item} with recycling options if available:"
    return generate_response(prompt)

# Function to analyze energy habits
def energy_saving(habits):
    prompt = f"Analyze these energy habits: {habits}. Provide 5 specific recommendations:"
    return generate_response(prompt)

# Function to collect citizen feedback
def collect_feedback(issue, category):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    feedback_data.append({"Timestamp": timestamp, "Issue": issue, "Category": category})
    df = pd.DataFrame(feedback_data)
    df.to_excel("feedback_data.xlsx", index=False)
    return f"✅ Feedback logged at {timestamp}\nCategory: {category}\nIssue: {issue}"

# Function to download feedback data
def download_feedback():
    if not feedback_data:
        return "No feedback data available to download"
    df = pd.DataFrame(feedback_data)
    filename = "smart_city_feedback.xlsx"
    df.to_excel(filename, index=False)
    return filename

# Function to get a green challenge
def green_challenge():
    challenges = [
        "Carry a reusable water bottle all day",
        "Meat-free meals for 24 hours",
        "Collect 10 pieces of litter in your neighborhood",
        "Calculate your carbon footprint",
        "Try a zero-waste lunch",
        "Use public transport or bike today",
        "Switch off all standby electronics overnight"
    ]
    challenge = random.choice(challenges)
    return f"🌱 TODAY'S CHALLENGE:\n{challenge}"

# Function to summarize policy documents
def summarize_policy(document):
    prompt = f"Summarize this policy document in 3-5 key points for citizens:\n{document}"
    return generate_response(prompt)

# Function to forecast resource usage
def forecast_kpi(file):
    if not file:
        return "⚠️ Please upload a CSV file first"
    try:
        df = pd.read_csv(file.name)
        if 'usage' not in df.columns:
            return "CSV must contain 'usage' column"
        avg = df['usage'].mean()
        forecast = avg * 1.1
        return f"📊 Water Usage Forecast\nCurrent average: {avg:.2f} units\nNext period projection: {forecast:.2f} units (+10%)"
    except Exception as e:
        return f"Error processing file: {str(e)}"

# Function to provide eco tips
def eco_tips(keyword):
    prompt = f"Generate 3 practical eco-tips about {keyword} for urban residents:"
    return generate_response(prompt)

# Function to detect anomalies in usage data
def detect_anomaly(file):
    if not file:
        return "⚠️ Please upload a CSV file first"
    try:
        df = pd.read_csv(file.name)
        if 'usage' not in df.columns:
            return "CSV must contain 'usage' column"
        mean = df['usage'].mean()
        std = df['usage'].std()
        anomalies = df[(df['usage'] > mean + 2*std) | (df['usage'] < mean - 2*std)]
        if anomalies.empty:
            return "✅ No anomalies detected in energy usage data"
        else:
            return (f"🚨 {len(anomalies)} ANOMALIES DETECTED!\n"
                   f"Average usage: {mean:.2f}\n"
                   f"Threshold: ±{2*std:.2f}\n"
                   f"Anomalous values:\n{anomalies['usage'].to_string(index=False)}")
    except Exception as e:
        return f"Error processing file: {str(e)}"

# Function for sustainability Q&A
def chat_assistant(question):
    prompt = f"Answer this city sustainability question knowledgeably and conversationally:\n{question}"
    return generate_response(prompt)



In [ ]:
# Create the enhanced Gradio Interface
with gr.Blocks(css=css) as app:
    with gr.Column():
        gr.Markdown("""
        <div class="header">
            <h1 style="font-size: 2.2em; margin-bottom: 10px;">🌿 Sustainable Smart City Assistant</h1>
            <p style="font-size: 1.1em;">Your AI Partner for Urban Sustainability</p>
        </div>
        """)

        with gr.Tabs():
            with gr.Tab("♻️ Waste Sorting"):
                gr.Markdown("### Get recycling and disposal guidance for any item")
                item_input = gr.Textbox(label="What item do you need to dispose?")
                waste_output = gr.Textbox(label="Disposal Instructions", lines=5, elem_classes=["output-text"])
                waste_btn = gr.Button("Get Instructions")
                waste_btn.click(waste_management, inputs=item_input, outputs=waste_output)

            with gr.Tab("💡 Energy Advisor"):
                gr.Markdown("### Optimize your energy consumption")
                habits_input = gr.Textbox(label="Describe your current energy habits")
                energy_output = gr.Textbox(label="Personalized Recommendations", lines=6, elem_classes=["output-text"])
                energy_btn = gr.Button("Analyze My Habits")
                energy_btn.click(energy_saving, inputs=habits_input, outputs=energy_output)

            with gr.Tab("📢 Citizen Feedback"):
                gr.Markdown("### Report urban issues to your local government")
                with gr.Row():
                    issue_input = gr.Textbox(label="Describe the issue", lines=3)
                    category_input = gr.Dropdown(["Water", "Waste", "Energy", "Transport", "Noise", "Other"], label="Category")
                feedback_output = gr.Textbox(label="Submission Status", lines=2, elem_classes=["output-text"])
                feedback_btn = gr.Button("Submit Report")
                dl_btn = gr.Button("Download Feedback Data")
                feedback_btn.click(collect_feedback, inputs=[issue_input, category_input], outputs=feedback_output)
                dl_btn.click(download_feedback, outputs=gr.File(label="Download"))

            with gr.Tab("🌎 Green Challenge"):
                gr.Markdown("### Daily sustainability mission")
                challenge_output = gr.Textbox(label="Today's Task", lines=4, elem_classes=["output-text"])
                challenge_btn = gr.Button("Get Today's Challenge")
                challenge_btn.click(green_challenge, outputs=challenge_output)

            with gr.Tab("📜 Policy Expert"):
                gr.Markdown("### Simplify complex urban policies")
                policy_input = gr.Textbox(label="Paste policy text", lines=5)
                policy_output = gr.Textbox(label="Key Takeaways", lines=6, elem_classes=["output-text"])
                policy_btn = gr.Button("Summarize")
                policy_btn.click(summarize_policy, inputs=policy_input, outputs=policy_output)

            with gr.Tab("📈 Resource Forecaster"):
                gr.Markdown("### Predict future resource usage")
                kpi_input = gr.File(label="Upload CSV with historical data", file_types=[".csv"], elem_classes=["upload-box"])
                kpi_output = gr.Textbox(label="Forecast Results", lines=4, elem_classes=["output-text"])
                kpi_btn = gr.Button("Generate Forecast")
                kpi_btn.click(forecast_kpi, inputs=kpi_input, outputs=kpi_output)

            with gr.Tab("✨ Eco Tips"):
                gr.Markdown("### Get customized sustainability advice")
                keyword_input = gr.Textbox(label="Enter topic (e.g., composting, EVs, solar)")
                tips_output = gr.Textbox(label="Actionable Tips", lines=5, elem_classes=["output-text"])
                tips_btn = gr.Button("Get Tips")
                tips_btn.click(eco_tips, inputs=keyword_input, outputs=tips_output)

            with gr.Tab("🔍 Usage Monitor"):
                gr.Markdown("### Detect irregularities in city resource usage")
                anomaly_input = gr.File(label="Upload utility data CSV", file_types=[".csv"], elem_classes=["upload-box"])
                anomaly_output = gr.Textbox(label="Analysis Results", lines=5, elem_classes=["output-text"])
                anomaly_btn = gr.Button("Scan for Anomalies")
                anomaly_btn.click(detect_anomaly, inputs=anomaly_input, outputs=anomaly_output)

            with gr.Tab("💬 Sustainability Q&A"):
                gr.Markdown("### Ask me anything about urban sustainability")
                chat_input = gr.Textbox(label="Your question", placeholder="How can my city reduce plastic waste?")
                chat_output = gr.Textbox(label="AI Response", lines=6, elem_classes=["output-text"])
                chat_btn = gr.Button("Ask the Expert")
                chat_btn.click(chat_assistant, inputs=chat_input, outputs=chat_output)



In [ ]:
# Launch the app
app.launch(server_name="0.0.0.0", server_port=7860)
